In [1]:
import h3
import os
import pandas as pd
import polars as pl
from datetime import date
from rocrate.rocrate import ROCrate
from rocrate.model import (
    Person,
    File,
    Dataset,
    ComputationalWorkflow,
    ContextEntity,
)

In [2]:
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 1000)

In [3]:
def get_file_size(path):
    file_stats = os.stat(path)
    size = file_stats.st_size
    if size >= 1024*1024:
        return f"{round(file_stats.st_size / (1024*1024), 2)}MB"
    elif size >= 1024:
        return f"{round(file_stats.st_size / 1024, 2)}KB"
    else:         
        return f"{file_stats.st_size}B"

## Read in Data

### Site Metadata

In [4]:
in_crate_sites = ROCrate("../../metastore/USGSGroundwaterSiteMetadata/")

In [5]:
file_uris= [file["@id"] for file in in_crate_sites.mainEntity.properties().get("hasParts", [])]
file_uris

['file:///data/extract/USGSGroundwaterSiteMetadata/400132075031001.rdb',
 'file:///data/extract/USGSGroundwaterSiteMetadata/400001075040301.rdb',
 'file:///data/extract/USGSGroundwaterSiteMetadata/400217075142101.rdb',
 'file:///data/extract/USGSGroundwaterSiteMetadata/395611075091301.rdb',
 'file:///data/extract/USGSGroundwaterSiteMetadata/395353075151501.rdb',
 'file:///data/extract/USGSGroundwaterSiteMetadata/395416075150301.rdb',
 'file:///data/extract/USGSGroundwaterSiteMetadata/395459075140501.rdb',
 'file:///data/extract/USGSGroundwaterSiteMetadata/395705075135901.rdb',
 'file:///data/extract/USGSGroundwaterSiteMetadata/400211075093701.rdb',
 'file:///data/extract/USGSGroundwaterSiteMetadata/400327075152201.rdb',
 'file:///data/extract/USGSGroundwaterSiteMetadata/400424075104901.rdb',
 'file:///data/extract/USGSGroundwaterSiteMetadata/400512075033401.rdb',
 'file:///data/extract/USGSGroundwaterSiteMetadata/400311075101301.rdb',
 'file:///data/extract/USGSGroundwaterSiteMetadata/

In [6]:
input_data_paths = [uri.replace("file://", "../..") for uri in file_uris]
input_data_paths

['../../data/extract/USGSGroundwaterSiteMetadata/400132075031001.rdb',
 '../../data/extract/USGSGroundwaterSiteMetadata/400001075040301.rdb',
 '../../data/extract/USGSGroundwaterSiteMetadata/400217075142101.rdb',
 '../../data/extract/USGSGroundwaterSiteMetadata/395611075091301.rdb',
 '../../data/extract/USGSGroundwaterSiteMetadata/395353075151501.rdb',
 '../../data/extract/USGSGroundwaterSiteMetadata/395416075150301.rdb',
 '../../data/extract/USGSGroundwaterSiteMetadata/395459075140501.rdb',
 '../../data/extract/USGSGroundwaterSiteMetadata/395705075135901.rdb',
 '../../data/extract/USGSGroundwaterSiteMetadata/400211075093701.rdb',
 '../../data/extract/USGSGroundwaterSiteMetadata/400327075152201.rdb',
 '../../data/extract/USGSGroundwaterSiteMetadata/400424075104901.rdb',
 '../../data/extract/USGSGroundwaterSiteMetadata/400512075033401.rdb',
 '../../data/extract/USGSGroundwaterSiteMetadata/400311075101301.rdb',
 '../../data/extract/USGSGroundwaterSiteMetadata/400055075122501.rdb',
 '../.

In [7]:
def get_header(path):
    with open(path, "r") as f:
        text = f.read()
        len_header = 0
        for line in text.split("\n"):
            if line[0] == "#":
                len_header += 1
            else:
                break
        return len_header

In [8]:
dfs = []
for input_path in input_data_paths:
    df = pd.read_table(input_path, header=get_header(input_path))
    dfs.append(df[1:])
site_df = pd.concat(dfs).reset_index(drop=True)
print(site_df.shape)
site_df.head()

(23, 42)


,agency_cd,site_no,station_nm,site_tp_cd,lat_va,long_va,dec_lat_va,dec_long_va,coord_meth_cd,coord_acy_cd,coord_datum_cd,dec_coord_datum_cd,district_cd,state_cd,county_cd,country_cd,land_net_ds,map_nm,map_scale_fc,alt_va,alt_meth_cd,alt_acy_va,alt_datum_cd,huc_cd,basin_cd,topo_cd,instruments_cd,construction_dt,inventory_dt,drain_area_va,contrib_drain_area_va,tz_cd,local_time_fg,reliability_cd,gw_file_cd,nat_aqfr_cd,aqfr_cd,aqfr_type_cd,well_depth_va,hole_depth_va,depth_src_cd,project_no
0,USGS,400132075031001,PH 1056,GW,400132,0750310,40.02555556,-75.05277778,M,S,NAD83,NAD83,42,42,101,US,NaN,FRANKFORD,24000,48,M,5,NAVD88,02040202,NaN,F,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNN,20140617,20150226,NaN,NaN,EST,Y,C,YY Y Y,N400PDMBRX,300WSCK,U,29,29,O,247600265
1,USGS,400001075040301,PH 1057,GW,400001,0750403,40.00027778,-75.0675,M,S,NAD83,NAD83,42,42,101,US,NaN,FRANKFORD,24000,16.00,M,5,NAVD88,02040202,NaN,G,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNN,20140618,20150226,NaN,NaN,EST,Y,C,YY Y Y,N100GLCIAL,112TREN,U,23,24,O,247600265
2,USGS,400217075142101,PH 540,GW,400217.5,0751418.1,40.03819444,-75.2383611,G,1,NAD83,NAD83,42,42,101,US,NaN,GERMANTOWN,24000,170,M,10,NGVD29,02040203,NaN,S,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNN,19480329,NaN,NaN,NaN,EST,Y,C,YYYYNYNN,N400PDMBRX,300WSCKO,NaN,127,NaN,NaN,NaN
3,USGS,395611075091301,PH 1059,GW,395611,0750913,39.93638889,-75.1536111,M,S,NAD83,NAD83,42,42,101,US,NaN,PHILADELPHIA,24000,25,M,5,NAVD88,02040202,NaN,NaN,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNN,20140625,20150226,NaN,NaN,EST,Y,C,YY Y Y,N100GLCIAL,112TREN,U,32.0,32.5,O,247600265
4,USGS,395353075151501,PH 1052,GW,395353.30,0751515.30,39.89813889,-75.25425,G,R,NAD83,NAD83,42,42,101,US,NaN,LANSDOWNE,24000,16,G,10,NAVD88,02040202,NaN,F,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNN,20110307,20120507,NaN,NaN,EST,Y,C,YY Y Y,S100NATLCP,112TREN,U,30.50,NaN,R,247600265


In [9]:
site_df["site_no"].value_counts()

site_no
400132075031001    1
400311075101301    1
400308074592201    1
400644074590801    1
395341075102101    1
395408075104001    1
395656075104401    1
395849075134201    1
400016075102801    1
400038075094601    1
400055075122501    1
400512075033401    1
400001075040301    1
400424075104901    1
400327075152201    1
400211075093701    1
395705075135901    1
395459075140501    1
395416075150301    1
395353075151501    1
395611075091301    1
400217075142101    1
400516075033201    1
Name: count, dtype: int64

### Site Observations

In [10]:
in_crate_obs = ROCrate("../../metastore/USGSGroundwaterObservations/")

In [11]:
file_uris= [file["@id"] for file in in_crate_obs.mainEntity.properties().get("hasParts", [])]
file_uris

['file:///data/extract/USGSGroundwaterObservations/400132075031001.rdb',
 'file:///data/extract/USGSGroundwaterObservations/400001075040301.rdb',
 'file:///data/extract/USGSGroundwaterObservations/400217075142101.rdb',
 'file:///data/extract/USGSGroundwaterObservations/395611075091301.rdb',
 'file:///data/extract/USGSGroundwaterObservations/395353075151501.rdb',
 'file:///data/extract/USGSGroundwaterObservations/395416075150301.rdb',
 'file:///data/extract/USGSGroundwaterObservations/395459075140501.rdb',
 'file:///data/extract/USGSGroundwaterObservations/395705075135901.rdb',
 'file:///data/extract/USGSGroundwaterObservations/400211075093701.rdb',
 'file:///data/extract/USGSGroundwaterObservations/400327075152201.rdb',
 'file:///data/extract/USGSGroundwaterObservations/400424075104901.rdb',
 'file:///data/extract/USGSGroundwaterObservations/400512075033401.rdb',
 'file:///data/extract/USGSGroundwaterObservations/400311075101301.rdb',
 'file:///data/extract/USGSGroundwaterObservations/

In [12]:
input_data_paths = [uri.replace("file://", "../..") for uri in file_uris]
input_data_paths

['../../data/extract/USGSGroundwaterObservations/400132075031001.rdb',
 '../../data/extract/USGSGroundwaterObservations/400001075040301.rdb',
 '../../data/extract/USGSGroundwaterObservations/400217075142101.rdb',
 '../../data/extract/USGSGroundwaterObservations/395611075091301.rdb',
 '../../data/extract/USGSGroundwaterObservations/395353075151501.rdb',
 '../../data/extract/USGSGroundwaterObservations/395416075150301.rdb',
 '../../data/extract/USGSGroundwaterObservations/395459075140501.rdb',
 '../../data/extract/USGSGroundwaterObservations/395705075135901.rdb',
 '../../data/extract/USGSGroundwaterObservations/400211075093701.rdb',
 '../../data/extract/USGSGroundwaterObservations/400327075152201.rdb',
 '../../data/extract/USGSGroundwaterObservations/400424075104901.rdb',
 '../../data/extract/USGSGroundwaterObservations/400512075033401.rdb',
 '../../data/extract/USGSGroundwaterObservations/400311075101301.rdb',
 '../../data/extract/USGSGroundwaterObservations/400055075122501.rdb',
 '../.

In [13]:
def get_header(path):
    with open(path, "r") as f:
        text = f.read()
        len_header = 0
        for line in text.split("\n"):
            if line[0] == "#":
                len_header += 1
            else:
                break
        return len_header

In [14]:
dfs = []
for input_path in input_data_paths:
    df = pd.read_table(input_path, header=get_header(input_path))
    dfs.append(df[1:])
obs_df = pd.concat(dfs).reset_index(drop=True)
print(obs_df.shape)
obs_df.head()

(8658, 16)


,agency_cd,site_no,site_tp_cd,lev_dt,lev_tm,lev_tz_cd,lev_va,sl_lev_va,sl_datum_cd,lev_status_cd,lev_agency_cd,lev_dt_acy_cd,lev_acy_cd,lev_src_cd,lev_meth_cd,lev_age_cd
0,USGS,400132075031001,GW,2014-08-14,14:55,UTC,NaN,28.91,NGVD29,1,USGS,m,2,S,V,A
1,USGS,400132075031001,GW,2014-08-14,14:55,UTC,NaN,27.74,NAVD88,1,USGS,m,2,S,V,A
2,USGS,400132075031001,GW,2014-08-14,14:55,UTC,20.26,NaN,NaN,1,USGS,m,2,S,V,A
3,USGS,400132075031001,GW,2014-09-15,16:36,UTC,NaN,28.86,NGVD29,1,USGS,m,2,S,V,A
4,USGS,400132075031001,GW,2014-09-15,16:36,UTC,NaN,27.69,NAVD88,1,USGS,m,2,S,V,A


## Conform Data

In [15]:
limited_obs_df = (
    obs_df[
        (obs_df["sl_datum_cd"] == "NAVD88")
    ]
    .drop(columns=["lev_va"])
)
limited_obs_df.head()

,agency_cd,site_no,site_tp_cd,lev_dt,lev_tm,lev_tz_cd,sl_lev_va,sl_datum_cd,lev_status_cd,lev_agency_cd,lev_dt_acy_cd,lev_acy_cd,lev_src_cd,lev_meth_cd,lev_age_cd
1,USGS,400132075031001,GW,2014-08-14,14:55,UTC,27.74,NAVD88,1,USGS,m,2,S,V,A
4,USGS,400132075031001,GW,2014-09-15,16:36,UTC,27.69,NAVD88,1,USGS,m,2,S,V,A
7,USGS,400132075031001,GW,2014-11-13,15:15,UTC,27.00,NAVD88,1,USGS,m,2,S,V,A
10,USGS,400132075031001,GW,2014-12-16,16:15,UTC,27.00,NAVD88,1,USGS,m,2,S,V,A
13,USGS,400132075031001,GW,2015-01-14,15:53,UTC,27.13,NAVD88,1,USGS,m,2,S,V,A


In [16]:
limited_obs_df.shape

(2886, 15)

In [17]:
site_df.shape

(23, 42)

In [18]:
out_df = (
    limited_obs_df
    .merge(
        site_df,
        on=["agency_cd", "site_no", "site_tp_cd",],
        how="left",
    )
)
print(out_df.shape)
out_df.head()

(2886, 54)


,agency_cd,site_no,site_tp_cd,lev_dt,lev_tm,lev_tz_cd,sl_lev_va,sl_datum_cd,lev_status_cd,lev_agency_cd,lev_dt_acy_cd,lev_acy_cd,lev_src_cd,lev_meth_cd,lev_age_cd,station_nm,lat_va,long_va,dec_lat_va,dec_long_va,coord_meth_cd,coord_acy_cd,coord_datum_cd,dec_coord_datum_cd,district_cd,state_cd,county_cd,country_cd,land_net_ds,map_nm,map_scale_fc,alt_va,alt_meth_cd,alt_acy_va,alt_datum_cd,huc_cd,basin_cd,topo_cd,instruments_cd,construction_dt,inventory_dt,drain_area_va,contrib_drain_area_va,tz_cd,local_time_fg,reliability_cd,gw_file_cd,nat_aqfr_cd,aqfr_cd,aqfr_type_cd,well_depth_va,hole_depth_va,depth_src_cd,project_no
0,USGS,400132075031001,GW,2014-08-14,14:55,UTC,27.74,NAVD88,1,USGS,m,2,S,V,A,PH 1056,400132,0750310,40.02555556,-75.05277778,M,S,NAD83,NAD83,42,42,101,US,NaN,FRANKFORD,24000,48,M,5,NAVD88,02040202,NaN,F,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNN,20140617,20150226,NaN,NaN,EST,Y,C,YY Y Y,N400PDMBRX,300WSCK,U,29,29,O,247600265
1,USGS,400132075031001,GW,2014-09-15,16:36,UTC,27.69,NAVD88,1,USGS,m,2,S,V,A,PH 1056,400132,0750310,40.02555556,-75.05277778,M,S,NAD83,NAD83,42,42,101,US,NaN,FRANKFORD,24000,48,M,5,NAVD88,02040202,NaN,F,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNN,20140617,20150226,NaN,NaN,EST,Y,C,YY Y Y,N400PDMBRX,300WSCK,U,29,29,O,247600265
2,USGS,400132075031001,GW,2014-11-13,15:15,UTC,27.00,NAVD88,1,USGS,m,2,S,V,A,PH 1056,400132,0750310,40.02555556,-75.05277778,M,S,NAD83,NAD83,42,42,101,US,NaN,FRANKFORD,24000,48,M,5,NAVD88,02040202,NaN,F,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNN,20140617,20150226,NaN,NaN,EST,Y,C,YY Y Y,N400PDMBRX,300WSCK,U,29,29,O,247600265
3,USGS,400132075031001,GW,2014-12-16,16:15,UTC,27.00,NAVD88,1,USGS,m,2,S,V,A,PH 1056,400132,0750310,40.02555556,-75.05277778,M,S,NAD83,NAD83,42,42,101,US,NaN,FRANKFORD,24000,48,M,5,NAVD88,02040202,NaN,F,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNN,20140617,20150226,NaN,NaN,EST,Y,C,YY Y Y,N400PDMBRX,300WSCK,U,29,29,O,247600265
4,USGS,400132075031001,GW,2015-01-14,15:53,UTC,27.13,NAVD88,1,USGS,m,2,S,V,A,PH 1056,400132,0750310,40.02555556,-75.05277778,M,S,NAD83,NAD83,42,42,101,US,NaN,FRANKFORD,24000,48,M,5,NAVD88,02040202,NaN,F,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNN,20140617,20150226,NaN,NaN,EST,Y,C,YY Y Y,N400PDMBRX,300WSCK,U,29,29,O,247600265


In [19]:
# Create GeoJSON Point Location for Sites
out_df["geometry"] = out_df.apply(lambda row: {"type": "Point", "coordinates": [float(row["dec_long_va"]), float(row["dec_lat_va"])]}, axis=1)

In [20]:
# Create Hashed Cells for Sites with Max Resolution
out_df["Cell15"] = out_df.apply(lambda row: h3.geo_to_h3(float(row["dec_lat_va"]), float(row["dec_long_va"]), 15), axis=1)

In [21]:
try:
    os.mkdir("../../data/common/USGSGroundwaterConformed")
except FileExistsError as e:
    pass

In [22]:
out_df.to_parquet("../../data/common/USGSGroundwaterConformed/USGSGroundwaterConformed.parquet")

## Create Metadata

Main Attributes

- name
- publisher {"@id": "#local_id"} or {"@id": "url"}
- creater {"@id": "#person"}
- license
- datePublished
- keywords (ex. "streets, aggregated features, map")

Contextual

- Creator
    - @type Person
    - name
 
- Publisher
    - @type Organization
    - name
    - url

Dataset

- Upstream Dataset(s)
- Output Dataset (Main Entity)
- Provenance of Producing Output Dataset
    - This file (Since it can't be processed all the way through with polars)

In [23]:
crate = ROCrate()

In [24]:
name = "USGS Groundwater Well Depth Data and Metadata for Sites in Philadelphia County, Pennsylvania"
short_name = "USGSGroundwaterConformed"

In [25]:
author = crate.add(
    Person(
        crate,
        "Schneider, Richard",
        properties={
            "name": "Richard Schneider",
        },
    )
)

In [26]:
project = crate.add(
    ContextEntity(
        crate,
        "https://github.com/rschneider98/data_curation",
        properties={
            "@type": "Organization",
            "name": "Data Curation Project",
            "url": "https://github.com/rschneider98/data_curation"
        },
    )
)

In [27]:
us_public_domain = crate.add(
    ContextEntity(
        crate,
        "U.S. Public Domain",
        properties={
            "@type": "License",
            "name": "U.S. Public Domain",
            "url": "http://www.usa.gov/publicdomain/label/1.0/",
        },
    )
)

In [28]:
date_published = date.today().isoformat()

In [29]:
crate.name = name
crate.publisher = project
crate.creator = author
crate.license = us_public_domain
crate.datePublished = date_published
crate.keywords = ["USGS", "US Geological Survey", "Groundwater", "Geometry"]

In [30]:
encoding_format = crate.add(
    ContextEntity(
        crate,
        "https://www.loc.gov/preservation/digital/formats/fdd/fdd000575.shtml",  # Apache Parquet does not exist in PRONOM
        properties={
            "@type": "Website",
            "name": "parquet",
        },
    )
)

In [31]:
# Dataset
out_file = crate.add (
    File(
        crate,
        source=f"file:///data/common/{short_name}/{short_name}.parquet",
        properties={
            "description": name,
            "contentSize": get_file_size(f"../../data/common/{short_name}/{short_name}.parquet"),
            "encodingFormat": [encoding_format.properties()["name"], {"@id": encoding_format.id}],
        }
    )
)

out_dataset = crate.add(
    Dataset(
        crate,
        source=f"file:///data/common/{short_name}",
        properties={
            "description": name,
            "hasParts": [
                {"@id": out_file.id},
            ]
        }
    )
)

### Consumed Datasets

In [32]:
# Upstream - Site Data
# in_crate_sites
sites_dataset = in_crate_sites.mainEntity

In [33]:
in_sites_dataset = crate.add(
    Dataset(
        crate,
        source=sites_dataset.source,
        properties={
            "description": sites_dataset.properties().get("description"),
            "name": in_crate_sites.name,
            "sameAs": str(in_crate_sites.source),
        }
    )
)

In [34]:
# Upstream - Shapes
# in_crate_shapes
obs_dataset = in_crate_obs.mainEntity

In [35]:
in_obs_dataset = crate.add(
    Dataset(
        crate,
        source=obs_dataset.source,
        properties={
            "description": obs_dataset.properties().get("description"),
            "name": in_crate_obs.name,
            "sameAs": str(in_crate_obs.source),
        }
    )
)

In [36]:
# Provenance - This file
# ContextualEntity of Type Computational Workflow

In [37]:
python_language = crate.add(
    ContextEntity(
        crate,
        identifier="python",
        properties={
            "@type": ["ComputerLanguage", "SoftwareApplication"],
            "name": "Python 3.12.2",
            "version": "3.12.2",
            "url": "https://www.python.org/downloads/release/python-3122/",
        },
    )
)

In [38]:
workflow = crate.add(
    ComputationalWorkflow(
        crate,
        source="file:///analysis/Conform/Conform - Groundwater Data.ipynb",
        properties={
            "author": {"@id": author.id},
            "programmingLanguage": {"@id": python_language.id},
        }
    )
)

In [39]:
action = crate.add(
    ContextEntity(
        crate,
        identifier="Conform - Groundwater Data",
        properties={
            "@type": "CreateAction",
            "name": "Conform Census Data",
            "instrument": {"@id": workflow.id},
            "result": {"@id": out_dataset.id},
        },
    )
)

In [40]:
# Main Entry
crate.mainEntity = out_dataset

### Store Metadata

In [41]:
crate.write(f"../../metastore/{short_name}/")